In [ ]:
#kaggle datasets
!pip install kaggle

In [ ]:
from google.colab import files

#retrieve uploaded file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving news.csv to news.csv
User uploaded file "news.csv" with length 30696129 bytes
mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d sid321axn/fakernewsdata/download/fakernewsdata.zip/1

 80% 33.0M/41.0M [00:00<00:00, 29.5MB/s]
100% 41.0M/41.0M [00:00<00:00, 44.4MB/s]


In [ ]:
!ls
!unzip fakernewsdata.zip
!ls

fakernewsdata.zip  sample_data
Archive:  fakernewsdata.zip
  inflating: fake_news.csv           
fake_news.csv  fakernewsdata.zip  sample_data


In [ ]:
import pandas as pd
df = pd.read_csv('fake_news.csv')

In [ ]:
df.head()

,Unnamed: 0,title,text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,TRUE
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,TRUE
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,TRUE
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,TRUE
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,TRUE


In [ ]:
import numpy as np
df =df.reindex(np.random.permutation(df.index))
df.head()

,Unnamed: 0,title,text,label
40925,40925,WOW! JILL STEIN’S ‘FIRESIDE CHAT’ Exposes Her ...,,FAKE
29304,29304,Donald Trump Just Said The DUMBEST Thing Abou...,Donald Trump made a pathetic appearance at Reg...,FAKE
16622,16622,EU's Tusk says 'good deal' or 'no deal' on Bre...,BRUSSELS (Reuters) - European Council Presiden...,TRUE
28554,28554,Rosie O’Donnell Writes Powerful Poem That Per...,If anyone knows what it s like to be attacked ...,FAKE
40764,40764,BOOM! Trump Drains Obama Swamp…Mandates All Am...,President-elect Donald J. Trump s transition s...,FAKE


In [ ]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df["label"] = df.label.apply(lambda  x:0 if x=='TRUE' else 1)
df.head()


,title,text,label
40925,WOW! JILL STEIN’S ‘FIRESIDE CHAT’ Exposes Her ...,,1
29304,Donald Trump Just Said The DUMBEST Thing Abou...,Donald Trump made a pathetic appearance at Reg...,1
16622,EU's Tusk says 'good deal' or 'no deal' on Bre...,BRUSSELS (Reuters) - European Council Presiden...,0
28554,Rosie O’Donnell Writes Powerful Poem That Per...,If anyone knows what it s like to be attacked ...,1
40764,BOOM! Trump Drains Obama Swamp…Mandates All Am...,President-elect Donald J. Trump s transition s...,1


In [ ]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['title'] = df['title'].str.replace(char, ' ')

In [ ]:
df['title']=df['title'].str.lower()
df.head()

,title,text,label
40925,wow jill stein’s ‘fireside chat’ exposes her ...,,1
29304,donald trump just said the dumbest thing abou...,Donald Trump made a pathetic appearance at Reg...,1
16622,eu s tusk says good deal or no deal on bre...,BRUSSELS (Reuters) - European Council Presiden...,0
28554,rosie o’donnell writes powerful poem that per...,If anyone knows what it s like to be attacked ...,1
40764,boom trump drains obama swamp…mandates all am...,President-elect Donald J. Trump s transition s...,1


In [ ]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 139kB/s 
     |████████████████████████████████| 6.8MB 39.8MB/s 
     |████████████████████████████████| 983kB 54.3MB/s 
     |████████████████████████████████| 266kB 54.1MB/s 
     |████████████████████████████████| 1.3MB 44.7MB/s 
     |████████████████████████████████| 1.1MB 51.4MB/s 
     |████████████████████████████████| 471kB 51.2MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 
     |████████████████████████████████| 2.9MB 47.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=53c68c2ed60425b80f869cb987a2f8fc94285c9147f5f8fee35b69e9dd117c17
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=1df0182bf9a228a18a57bc3e79787c3f8df61257f246ed0ecdec10d2ea7be265
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import ktrain
from ktrain import text
import numpy as np
from sklearn.model_selection import train_test_split

categories = ['TRUE', 'FAKE']

y = df['label'].values
X = df['title']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.30, random_state=0)

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=categories,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
preprocessing train...
language: en


preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 31428 samples
31428/31428 [==============================] - 2782s 89ms/sample - loss: 0.1111 - accuracy: 0.9570


We can use the learner.validate method to test our model against the Test set.

In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=categories)

              precision    recall  f1-score   support

        TRUE       0.98      0.99      0.98      6425
        FAKE       0.99      0.98      0.98      7045

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



array([[6356,   69],
       [ 143, 6902]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['TRUE', 'FAKE']